In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from utils import calculate_accuracy  # From utils.py


In [5]:
# Load pretrained model
num_classes = 120
model = models.resnet18(pretrained=True)

# Freeze base layers
for param in model.parameters():
    param.requires_grad = False

# Replace classifier head
model.fc = nn.Linear(model.fc.in_features, num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


In [7]:
def train_one_epoch(model, dataloader, optimizer, criterion):
    model.train()
    running_loss, correct = 0.0, 0

    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        correct += (outputs.argmax(1) == labels).sum().item()

    return running_loss / len(dataloader), correct / len(dataloader.dataset)

def validate(model, dataloader, criterion):
    model.eval()
    running_loss, correct = 0.0, 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()

    return running_loss / len(dataloader), correct / len(dataloader.dataset)


In [13]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Define image transformations (e.g., normalization and augmentation)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Mean and std values for ImageNet
])

# Load the datasets (replace with your actual dataset path)
train_dataset = datasets.ImageFolder(root=r"C:\Users\kulde\OneDrive\Desktop\final project\train", transform=transform)
test_dataset = datasets.ImageFolder(root=r"C:\Users\kulde\OneDrive\Desktop\final project\test", transform=transform)

# Create DataLoader for both training and validation
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Now you can use train_loader and val_loader in your training loop


FileNotFoundError: Couldn't find any class folder in C:\Users\kulde\OneDrive\Desktop\final project\train.

In [8]:
best_acc = 0.0
epochs = 10

for epoch in range(epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, val_loader, criterion)

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")
    print(f"Val Loss:   {val_loss:.4f}, Accuracy: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_dog_breed_model.pt")
        print("Model saved!")


NameError: name 'train_loader' is not defined

In [ ]:
from utils import load_image_for_prediction

model.load_state_dict(torch.load("best_dog_breed_model.pt"))
model.eval()

image_path = "train/your_image.jpg"  # replace with actual image path
img_tensor = load_image_for_prediction(image_path, transform).to(device)

with torch.no_grad():
    outputs = model(img_tensor)
    _, pred = torch.max(outputs, 1)
    predicted_breed = label_encoder.inverse_transform([pred.item()])[0]
    print(f"Predicted Breed: {predicted_breed}")
